In [2]:
import re
import urllib.request
import zipfile
import nltk
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="resource/ted_en-20160408.xml")

('resource/ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fd7f4e3fe10>)

In [4]:
targetXML = open('resource/ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 content 블록 내용만 가져온다
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 토해 content 중간에 등장하는 (Audio), (Laughter)등의 배경음 제거
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해 nltk를 이용하여 문장 토큰화
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

# 각문장에 대해 nltk를 이용하여토큰화 수행
result = [word_tokenize(sentence) for sentence in normalized_text]

In [5]:
print('총 샘플의 개수: {}'.format(len(result)))
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

총 샘플의 개수: 273424
['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


In [6]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

# vector_size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
# window = 컨텍스트 윈도우 크기
# min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
# workers = 학습을 위한 프로세스 수
# sg = 0은 CBOW, 1은 Skip-gram.

In [7]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8496203422546387), ('guy', 0.8336416482925415), ('lady', 0.7595507502555847), ('boy', 0.7484878897666931), ('girl', 0.7384111285209656), ('gentleman', 0.7333728075027466), ('soldier', 0.7321121692657471), ('kid', 0.6867445707321167), ('poet', 0.6864503622055054), ('philosopher', 0.6553515195846558)]


In [8]:
model.wv.save_word2vec_format('resource/word-vector-model/eng_w2v') # 모델 저장

In [9]:
loaded_model = KeyedVectors.load_word2vec_format('resource/word-vector-model/eng_w2v')

In [10]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8496203422546387), ('guy', 0.8336416482925415), ('lady', 0.7595507502555847), ('boy', 0.7484878897666931), ('girl', 0.7384111285209656), ('gentleman', 0.7333728075027466), ('soldier', 0.7321121692657471), ('kid', 0.6867445707321167), ('poet', 0.6864503622055054), ('philosopher', 0.6553515195846558)]
